In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import pickle
import time


In [2]:
from itertools import product
from math import sqrt
from math import pow

import gurobipy as gp
from gurobipy import GRB

In [ ]:
m = gp.Model('led_location')
a = m.addVars(1, vtype=GRB.CONTINUOUS)
m.setObjective(a[0], GRB.MINIMIZE)
b=m.addConstr(a[0]-2<=0)
m.update()
m.optimize()

In [3]:
print(product(range(3),range(4)))

In [4]:
# 밝기정보 불러오기
with open('data.pickle', 'rb') as f:
    data = pickle.load(f) 

In [5]:
# 변수 설정
idx = list(data)
num_led = len(idx)
print(len(idx))
penalty = 0.0000000001
length = 3000
bright = 70*101*101/100000

3111


In [6]:
data2=[]
for i in range(num_led):
    data2.append(data[idx[i]].reshape(101*101))
data2 = np.asarray(data2)
del data

In [7]:
data3=[]
for i in range(num_led):
    data3.append(data2[i]*10**5)
data3 = np.asarray(data3)

In [ ]:
from skimage.io import imshow

In [ ]:
# 유니폼 목적식, 개수 제약조건, 시간제약
solution = []
timeLimit = 10800

for k in [75]:
    for penalty in range(1850,1700,-1):
        bright = k*101*101/100000*10**5
        
#         penalty = j
        start = time.time()
        m = gp.Model('led_location')
        m.setParam('TimeLimit', timeLimit)

        select = m.addVars(num_led, vtype=GRB.BINARY, name='Select')

        a = m.addVars(2, vtype=GRB.CONTINUOUS)

        m.setObjective(a[0]-a[1], GRB.MINIMIZE) # 목적식 산포 최소화

        m.addConstr(select.sum()<=penalty) # 제약식1 LED 개수 제한 (for문 통해 점점 줄이며 pareto 경계 구함.)
        m.addConstrs(gp.quicksum(select[i]*data3[i][j] for i in range(num_led))<=a[0] for j in range(10201)) # a[0]가 최대 밝기를 나타나게 해주는 제약식
          
        m.addConstrs(a[1]<=gp.quicksum(select[i]*data3[i][j]for i in range(num_led)) for j in range(10201)) # a[1]이 최소 밝기가 되게 하는 제약식
        m.addConstr(gp.quicksum(select[i]*data3[i][j] for i in range(num_led) for j in range(10201))>=bright) # 전체 픽셀 밝기 합이 75 이상이 되게하는 제약식
    
        m.update()
        m.optimize()
        print(time.time()-start)
        solution.append(select)
        temp=[]
        for i in range(len(select)):
            temp.append(select[i].x)
        with open(str(penalty)+'_solution.pickle', 'wb') as f: # 도출된 해를 led개수_solution.pickle이라는 파일명으로 저장.(led 개수마다 하나씩 해가 나옴.)
            pickle.dump(temp, f)

Changed value of parameter TimeLimit to 10800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 20404 rows, 3113 columns and 54541854 nonzeros
Model fingerprint: 0xff9d8ddf
Variable types: 2 continuous, 3111 integer (3111 binary)
Coefficient statistics:
  Matrix range     [2e-06, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 8e+05]
Presolve removed 0 rows and 0 columns (presolve time = 7s) ...
Presolve removed 0 rows and 0 columns (presolve time = 36s) ...
Presolve removed 0 rows and 0 columns (presolve time = 89s) ...
Presolve added 3 rows and 3 columns
Presolve time: 88.54s
Presolved: 20407 rows, 3116 columns, 54541676 nonzeros
Variable types: 2 continuous, 3114 integer (3111 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5750437e+02   0.000000e+00   1.641326e+06    131s
    2417    7.9314462e+

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\yoons\anaconda3\lib\site-packages\ipykernel\iostream.py", line 387, in write
    if self.echo is not None:
KeyboardInterrupt


   15931    5.3240664e-01   2.475606e+00   0.000000e+00    572s
   16253    5.3163037e-01   3.216267e+00   0.000000e+00    578s
   16416    5.3134527e-01   2.826221e+00   0.000000e+00    581s
   16742    5.3102909e-01   1.401417e-01   0.000000e+00    587s
   16941    5.3097299e-01   0.000000e+00   0.000000e+00    591s
   16941    5.3097299e-01   0.000000e+00   0.000000e+00    593s

Root relaxation: objective 5.309730e-01, 16941 iterations, 477.34 seconds
Total elapsed time = 596.73s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.53097    0  647          -    0.53097      -     -  600s
H    0     0                     122.8269194    0.53097   100%     -  651s


KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\yoons\anaconda3\lib\site-packages\ipykernel\iostream.py", line 352, in flush
    if not evt.wait(self.flush_timeout):
  File "C:\Users\yoons\anaconda3\lib\threading.py", line 552, in wait
    signaled = self._cond.wait(timeout)
  File "C:\Users\yoons\anaconda3\lib\threading.py", line 300, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt: 


     0     0    0.53097    0  647  122.82692    0.53097   100%     -  659s
